# Surprise- Simple Python Recommendation System Egine
a.k.a штука для коллаборативной фильтрации, где реккомендашки строятся  на базе поведения пользователя

In [ ]:
!pip uninstall numpy surprise -y
!pip install "numpy<2"
!pip install surprise --upgrade

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


  Using cached surprise-0.1-py2.py3-none-any.whl.metadata (327 bytes)
  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2469555 sha256=2bb31c5f0d25814f955455128effd5f3f15cb60ceb4c67214e6881cf631dcf6a
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [ ]:
!pip install wget

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9685 sha256=d186ac4d5cb584a875ccb41a420dede8bada1fd5b385d921b09b1ef6b7ab7b3c
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [ ]:
import wget

In [ ]:
import pandas as pd
import numpy as np
from surprise import accuracy, Dataset, Reader
from surprise import (BaselineOnly, CoClustering, KNNBaseline, KNNBasic,
                     KNNWithMeans, KNNWithZScore, NormalPredictor, NMF,
                     SlopeOne, SVD, SVDpp)
from surprise.model_selection import GridSearchCV, train_test_split

In [ ]:
dataset ='ml-1m'
url = f'https://files.grouplens.org/datasets/movielens/{dataset}.zip'
wget.download(url, 'MovieLens.zip')

'MovieLens (1).zip'

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile("MovieLens.zip", "r") as zip_ref:
  zip_ref.extractall()

In [ ]:
users =pd.read_csv(f'./{dataset}/users.dat', sep ='::', names = ['userID', 'gender', 'age', 'occupation', 'zip-code'], engine = 'python')
users.name = 'users'
movies = pd.read_csv(f'./{dataset}/movies.dat', sep='::',
                     names = ['movieId', 'title', 'genres'], encoding='latin-1', engine='python')
movies.name = 'movies'
ratings = pd.read_csv(f'./{dataset}/ratings.dat', sep='::',
                      names = ['userId', 'movieId', 'rating', 'timestamp'], engine='python')
ratings.name = 'ratings'

In [ ]:
def dubli(dataset) -> None:
  print(dataset.name)
  dataset.info()
  print(f'Дубли: {dataset.duplicated().sum()}')
  print()

In [ ]:
dubli(users)
dubli(movies)
dubli(ratings)

users
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   userID      6040 non-null   int64 
 1   gender      6040 non-null   object
 2   age         6040 non-null   int64 
 3   occupation  6040 non-null   int64 
 4   zip-code    6040 non-null   object
dtypes: int64(3), object(2)
memory usage: 236.1+ KB
Дубли: 0

movies
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  3883 non-null   int64 
 1   title    3883 non-null   object
 2   genres   3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB
Дубли: 0

ratings
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     -----------

In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [ ]:
movies_with_ratings = movies.merge(ratings, on = 'movieId').reset_index(drop = True)
movies_with_ratings.dropna(inplace = True)

In [ ]:
movies_with_ratings

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474
...,...,...,...,...,...,...
1000204,3952,"Contender, The (2000)",Drama|Thriller,5812,4,992072099
1000205,3952,"Contender, The (2000)",Drama|Thriller,5831,3,986223125
1000206,3952,"Contender, The (2000)",Drama|Thriller,5837,4,1011902656
1000207,3952,"Contender, The (2000)",Drama|Thriller,5927,1,979852537


In [ ]:
dataset = pd. DataFrame({
    'uid':movies_with_ratings.userId,
    'iid':movies_with_ratings.title,
    'rating': movies_with_ratings.rating}
)

In [ ]:
dataset

,uid,iid,rating
0,1,Toy Story (1995),5
1,6,Toy Story (1995),4
2,8,Toy Story (1995),4
3,9,Toy Story (1995),5
4,10,Toy Story (1995),5
...,...,...,...
1000204,5812,"Contender, The (2000)",4
1000205,5831,"Contender, The (2000)",3
1000206,5837,"Contender, The (2000)",4
1000207,5927,"Contender, The (2000)",1


In [ ]:
dataset.isnull().sum()

,0
uid,0
iid,0
rating,0


In [ ]:
reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
data = Dataset.load_from_df(dataset, reader)

In [ ]:
trainset, testset = train_test_split(data, test_size = .2, random_state = 21)

In [ ]:
models =[SVD(), SVDpp(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]
models_name = ['SVD', 'SVDpp', 'SlopeOne', 'NMF', "NormalPredictor", 'KNNBaseline', 'KNNBasic', "KNNWithMeans", "KNNWithZScore", "BaselineOnly", "CoClustering"]

In [ ]:
for idx, model in enumerate(models):
  model.fit(trainset)
  print(f'Модель{models_name[idx]}:rmse = {accuracy.rmse(model.test(testset), verbose=False)}')

МодельSVD:rmse = 0.8711425092136628
МодельSVDpp:rmse = 0.8593693017046775
МодельSlopeOne:rmse = 0.9145011529044181
МодельNMF:rmse = 1.5035453979012878
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
МодельNormalPredictor:rmse = 0.8921297881542896
Computing the msd similarity matrix...
Done computing similarity matrix.
МодельKNNBaseline:rmse = 0.9192505209922023
Computing the msd similarity matrix...
Done computing similarity matrix.
МодельKNNBasic:rmse = 0.9265590995828431
Computing the msd similarity matrix...
Done computing similarity matrix.
МодельKNNWithMeans:rmse = 0.9278355686797725
Estimating biases using als...
МодельKNNWithZScore:rmse = 0.9056271143463166
МодельBaselineOnly:rmse = 0.9122479143218802


In [ ]:
model = CoClustering()
model.fit(trainset)
predictions = model.test(testset)
print(f'Модель CoClustering: rmse = {accuracy.rmse(predictions, verbose=False)}')

Модель CoClustering: rmse = 0.9118669353053739


## Улучшим параметры

In [ ]:
params = {
    'n_factors': [10, 20, 30, 50, 100],
    'n_epochs': [5, 10, 20, 30, 50],
    'lr_all': [0.002, 0.005, 0.007, 0.01],
    'reg_all':[0.02, 0.08, 0.4, 0.6],
    'random_state': [21]
}

In [ ]:
grid_search = GridSearchCV(SVD, params, measures = ['rmse'], cv =5, refit = True)
grid_search.fit(data)

algo = grid_search.best_estimator['rmse']
algo.fit(trainset)

predictions = algo.test(testset)
accur_score = accuracy.rmse(predictions, verbose = True)

In [ ]:
print(f"Лучшие параметры:{grid_search.best_params['rmse']}")

In [ ]:
model = SVDpp(n_factors=100, n_epochs=50, lr_all=0.01, reg_all=0.08, random_state=21)
model.fit(trainset)
print(f"Лучшая модель с rmse:{accur_score.rmse(model.test(testset), verbose = False)}")

Подбор оптимальных параметров моделей с помощью GridSearchCV позволи добиться показателей rmse:

    SVD: rmse = 0.8927
    SVDpp: rmse = 0.8919
